In [5]:
import tensorflow as tf
import tensorflow_hub as hub
from PIL import Image
from PIL import ImageOps
import tempfile
from six.moves.urllib.request import urlopen
from six import BytesIO

In [6]:
module_handle = "https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1"
model = hub.load(module_handle)

In [7]:
model.signatures.keys()
detector = model.signatures['default']

In [8]:
def download_and_resize_image(url, new_width=256, new_height=256):

  _, filename = tempfile.mkstemp(suffix='.jpg')

  response = urlopen(url)

  image_data = response.read()

  image_data = BytesIO(image_data)

  pil_image = Image.open(image_data)

  pil_image = ImageOps.fit(pil_image, (new_width, new_height), Image.ANTIALIAS)

  pil_image_rgb = pil_image.convert("RGB")

  pil_image_rgb.save(filename, format = "JPEG", quality=90)

  print("Image downloaded to %s." % filename)

  return filename

In [9]:
# You can choose a different URL that points to an image of your choice
image_url = "https://upload.wikimedia.org/wikipedia/commons/f/fb/20130807_dublin014.JPG"

downloaded_image_path = download_and_resize_image(image_url, 3872, 2592)

<ipython-input-8-bda6e352a75f>:13: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  pil_image = ImageOps.fit(pil_image, (new_width, new_height), Image.ANTIALIAS)


Image downloaded to /tmp/tmp6cvutdne.jpg.


In [10]:
def load_img(path):
  img = tf.io.read_file(path)

  img = tf.image.decode_jpeg(img, channels=3)

  return img

In [13]:
def run_detector(detector, path):
  img = load_img(path)

  converted_img = tf.image.convert_image_dtype(img, tf.float32)[tf.newaxis, ...]

  result = detector(converted_img)

  result = {key:value.numpy() for key, value in result.items()}

  print("Found %d objects." % len(result["detection_scores"]))
  print(result["detection_scores"])
  print(result["detection_class_entities"])
  print(result["detection_boxes"])

In [14]:
run_detector(detector, downloaded_image_path)

Found 100 objects.
[0.6544857  0.6114536  0.604229   0.5926326  0.59218603 0.58049154
 0.5514059  0.49466917 0.47515702 0.4734224  0.43996006 0.41485107
 0.40629694 0.39828914 0.397652   0.37620965 0.3727938  0.36574715
 0.3526071  0.33274624 0.30428708 0.2727659  0.2686493  0.25777096
 0.25290594 0.24612105 0.23403853 0.20342878 0.18229379 0.18045738
 0.17571312 0.16435076 0.15849952 0.15666048 0.15470827 0.15452749
 0.1492489  0.1334067  0.12948263 0.12649718 0.12044212 0.11767308
 0.11356075 0.11114758 0.11100268 0.10914917 0.10604051 0.08940531
 0.08598229 0.08280209 0.08104504 0.07806076 0.0776033  0.07628641
 0.07546869 0.07444178 0.07427166 0.07204837 0.07177538 0.07102203
 0.07032707 0.06809692 0.06304511 0.06285931 0.06270926 0.06223952
 0.05882177 0.05815045 0.05795773 0.05787582 0.05462382 0.05274325
 0.05133715 0.04826548 0.0470842  0.04682969 0.04495224 0.04405145
 0.0436071  0.04113451 0.04109946 0.03968579 0.03935    0.03912786
 0.03879518 0.03878603 0.03739649 0.0360693